# In this notebook I am using Logistic Regression
***
The model will classify new clients for a large Telecom and whether they will Churn ro not. How loayl they will be to the company. 


In [1]:
## I like to load all the libraries at the top so I can always check what I have or might need

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, Lasso, Ridge
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder



In [3]:
### Upload the csv file there are several ways to do that on Colab, 
### easiest is to uplode it to your files on the side bar or write the below code

from google.colab import files
data = files.upload()

Saving GDN clients.xlsx to GDN clients.xlsx


In [6]:
gdn_df = pd.read_excel('GDN clients.xlsx')

pd.set_option('display.max_columns', None) ## to see all columns
print(gdn_df.shape)
gdn_df.head(5)

(7043, 33)


,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [7]:
## Make a dataframe conatining just the columns we are going to need

columns_needed = ['Zip Code', 'Latitude','Longitude', 
                  'Gender', 'Senior Citizen','Phone Service',              
                  'Multiple Lines', 'Internet Service', 
                  'Device Protection','Streaming TV','Streaming Movies', 
                  'Contract', 'Payment Method', 'Tenure Months', 
                  'Monthly Charges', 'Total Charges' ,'Churn Label' ]


#gdn_df = gdn_df.reindex(columns=columns_needed)


gdn_df = gdn_df[columns_needed]

gdn_df.head(5)

 

,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Phone Service,Multiple Lines,Internet Service,Device Protection,Streaming TV,Streaming Movies,Contract,Payment Method,Tenure Months,Monthly Charges,Total Charges,Churn Label
0,90003,33.964131,-118.272783,Male,No,Yes,No,DSL,No,No,No,Month-to-month,Mailed check,2,53.85,108.15,Yes
1,90005,34.059281,-118.307420,Female,No,Yes,No,Fiber optic,No,No,No,Month-to-month,Electronic check,2,70.70,151.65,Yes
2,90006,34.048013,-118.293953,Female,No,Yes,Yes,Fiber optic,Yes,Yes,Yes,Month-to-month,Electronic check,8,99.65,820.5,Yes
3,90010,34.062125,-118.315709,Female,No,Yes,Yes,Fiber optic,Yes,Yes,Yes,Month-to-month,Electronic check,28,104.80,3046.05,Yes
4,90015,34.039224,-118.266293,Male,No,Yes,Yes,Fiber optic,Yes,Yes,Yes,Month-to-month,Bank transfer (automatic),49,103.70,5036.3,Yes


In [8]:
### Show the unique values in each column 

categorical_column_list = ['Gender', 'Senior Citizen', 'Phone Service',              
             'Multiple Lines', 'Internet Service', 'Contract', 
             'Device Protection', 'Streaming TV', 'Streaming Movies',
             'Payment Method', 'Churn Label']

unique_values_list = list()

for x in categorical_column_list:
  values = gdn_df[x].unique() 
  unique_values_list.append(values)


unique_values_list


[array(['Male', 'Female'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Yes', 'No'], dtype=object),
 array(['No', 'Yes', 'No phone service'], dtype=object),
 array(['DSL', 'Fiber optic', 'No'], dtype=object),
 array(['Month-to-month', 'Two year', 'One year'], dtype=object),
 array(['No', 'Yes', 'No internet service'], dtype=object),
 array(['No', 'Yes', 'No internet service'], dtype=object),
 array(['No', 'Yes', 'No internet service'], dtype=object),
 array(['Mailed check', 'Electronic check', 'Bank transfer (automatic)',
        'Credit card (automatic)'], dtype=object),
 array(['Yes', 'No'], dtype=object)]

In [9]:
### I am endoing the object (string) containing columns 
### As you see below with Gender for example, from male and female
### becomes 0s and 1s. 


le = LabelEncoder()


for x in categorical_column_list:
  gdn_df[x] = le.fit_transform(gdn_df[x])


print(gdn_df.Gender.value_counts())


gdn_df.head(5)

1    3555
0    3488
Name: Gender, dtype: int64


,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Phone Service,Multiple Lines,Internet Service,Device Protection,Streaming TV,Streaming Movies,Contract,Payment Method,Tenure Months,Monthly Charges,Total Charges,Churn Label
0,90003,33.964131,-118.272783,1,0,1,0,0,0,0,0,0,3,2,53.85,108.15,1
1,90005,34.059281,-118.307420,0,0,1,0,1,0,0,0,0,2,2,70.70,151.65,1
2,90006,34.048013,-118.293953,0,0,1,2,1,2,2,2,0,2,8,99.65,820.5,1
3,90010,34.062125,-118.315709,0,0,1,2,1,2,2,2,0,2,28,104.80,3046.05,1
4,90015,34.039224,-118.266293,1,0,1,2,1,2,2,2,0,0,49,103.70,5036.3,1


In [10]:
### Checking in it was dall done properly

categorical_column_list_labelled = ['Gender', 'Senior Citizen', 'Phone Service',              
             'Multiple Lines', 'Internet Service', 'Contract', 
             'Device Protection', 'Streaming TV', 'Streaming Movies',
             'Payment Method', 'Churn Label', 'Monthly Charges', 'Total Charges']

unique_values_list_labelled = list()

for x in categorical_column_list_labelled:
  values = gdn_df[x].unique() 
  unique_values_list_labelled.append(values)


unique_values_list_labelled


[array([1, 0]),
 array([0, 1]),
 array([1, 0]),
 array([0, 2, 1]),
 array([0, 1, 2]),
 array([0, 2, 1]),
 array([0, 2, 1]),
 array([0, 2, 1]),
 array([0, 2, 1]),
 array([3, 2, 0, 1]),
 array([1, 0]),
 array([ 53.85,  70.7 ,  99.65, ..., 108.35,  63.1 ,  78.7 ]),
 array([108.15, 151.65, 820.5, ..., 7362.9, 346.45, 6844.5], dtype=object)]

In [11]:
## We can see in the print out above that Total charges column is an object so we will convert it to float next


gdn_df['Total Charges'] = pd.to_numeric(gdn_df['Total Charges'], errors='coerce')

gdn_df.groupby('Total Charges').size().sort_values(ascending=False)


#dataFrame.groupby('Reg_Price').size().sort_values(ascending=False)

Total Charges
20.20      11
19.75       9
19.90       8
20.05       8
19.65       8
           ..
927.15      1
927.10      1
926.20      1
926.00      1
8684.80     1
Length: 6530, dtype: int64

In [12]:
### Making sure there is no NaN values in our table before we train it 

gdn_df.fillna(0)

,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Phone Service,Multiple Lines,Internet Service,Device Protection,Streaming TV,Streaming Movies,Contract,Payment Method,Tenure Months,Monthly Charges,Total Charges,Churn Label
0,90003,33.964131,-118.272783,1,0,1,0,0,0,0,0,0,3,2,53.85,108.15,1
1,90005,34.059281,-118.307420,0,0,1,0,1,0,0,0,0,2,2,70.70,151.65,1
2,90006,34.048013,-118.293953,0,0,1,2,1,2,2,2,0,2,8,99.65,820.50,1
3,90010,34.062125,-118.315709,0,0,1,2,1,2,2,2,0,2,28,104.80,3046.05,1
4,90015,34.039224,-118.266293,1,0,1,2,1,2,2,2,0,0,49,103.70,5036.30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,92285,34.341737,-116.539416,0,0,1,0,2,1,1,1,2,0,72,21.15,1419.40,0
7039,92301,34.667815,-117.536183,1,0,1,2,0,2,2,2,1,3,24,84.80,1990.50,0
7040,92304,34.559882,-115.637164,0,0,1,2,1,2,2,2,1,1,72,103.20,7362.90,0
7041,92305,34.167800,-116.864330,0,0,0,1,0,0,0,0,0,2,11,29.60,346.45,0


In [13]:
gdn_df['Churn Label'].value_counts()

0    5174
1    1869
Name: Churn Label, dtype: int64

In [14]:
'NaN' in gdn_df

False

In [18]:
### Splitting the data into training and testing sets 


features = gdn_df.columns[:-1]
#features = gdn_df.drop('Churn Label', axis=1)
Y = gdn_df['Churn Label']

X_train, X_test, y_train, y_test = train_test_split(gdn_df[features], Y, test_size=0.3)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(4930, 16)
(4930,)
(2113, 16)
(2113,)


False

In [22]:
### Making sure there is not nan and infinite values

X_train_clean = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_test_clean = X_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [49]:
sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train_clean)
X_test_sc = sc.transform(X_test_clean)

X_train_sc

array([[-1.74617064, -0.88452915,  0.42930492, ...,  0.66639091,
        -1.47369848, -0.58238568],
       [-0.69772643, -1.10448461,  1.60933251, ..., -0.87984663,
        -1.48030812, -0.91464907],
       [ 0.64934919,  0.67379844, -1.16641971, ..., -0.5950134 ,
         0.66782509, -0.37639639],
       ...,
       [ 1.40940387,  1.96972725, -0.34879056, ...,  0.70708137,
        -1.4604792 , -0.53994391],
       [-1.14459545, -0.76590029,  0.52946334, ..., -1.16467986,
         0.17871168, -0.89117251],
       [-0.63843138, -0.92212952,  1.25322888, ..., -1.24606078,
        -1.48691777, -0.99149538]])

In [50]:
## Fitting the scaled and encoded data 

lr = LogisticRegression(solver='liblinear').fit(X_train_sc, y_train)
y_predicted = lr.predict(X_test_sc)


print(y_predicted.shape)

(2113,)


In [55]:
results_df = pd.DataFrame(y_predicted)
results_df.columns = ['Churn Prediction']
results_df['Churn Prediction'].unique()
results_df['Churn actual'] = gdn_df['Churn Label']

results_df.tail(5)

,Churn Prediction,Churn actual
2108,0,0
2109,0,0
2110,1,0
2111,0,0
2112,0,0


In [137]:
print(r2_score(y_predicted, y_test))
print(np.sqrt(mean_squared_error(y_predicted, y_test)))

-0.18551508396869254
0.4578804643575895


In [56]:
predict_probability = lr.predict_proba(X_test_clean)
predict_probability

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [38]:
gdn_df.head(5)

,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Phone Service,Multiple Lines,Internet Service,Device Protection,Streaming TV,Streaming Movies,Contract,Payment Method,Tenure Months,Monthly Charges,Total Charges,Churn Label
0,90003,33.964131,-118.272783,1,0,1,0,0,0,0,0,0,3,2,53.85,108.15,1
1,90005,34.059281,-118.307420,0,0,1,0,1,0,0,0,0,2,2,70.70,151.65,1
2,90006,34.048013,-118.293953,0,0,1,2,1,2,2,2,0,2,8,99.65,820.50,1
3,90010,34.062125,-118.315709,0,0,1,2,1,2,2,2,0,2,28,104.80,3046.05,1
4,90015,34.039224,-118.266293,1,0,1,2,1,2,2,2,0,0,49,103.70,5036.30,1


In [135]:
### Let's check how accurate our model is!

check_client_number = 500

# We have to reshape the arrays first to be the same as when model was trained

reshaped_y = y_test.values.reshape(1,-1) 
reshaped_x = X_test_sc[check_client_number,:].reshape(1,-1)

print(f' The predicted label is {lr.predict(reshaped_x)}, the actual Churn value is {reshaped_y[0,check_client_number]} ')

#sample_reshaped = X_test_clean.iloc[check_client_number, :].values.reshape(1,-1)
#print(f' The predicted label is {lr.predict(sample_reshaped)}, the actual Churn value is {reshaped_y[0,check_client_number]} ')





print(y_test.value_counts())

np.unique(y_predicted, return_counts=True)

 The predicted label is [1], the actual Churn value is 1 
0    1547
1     566
Name: Churn Label, dtype: int64


(array([0, 1]), array([1628,  485]))

In [161]:
from sklearn.metrics import accuracy_score

accuracy_score = np.round(accuracy_score(y_test, y_predicted),2)
print(f' The accuracy of our model is {accuracy_score}')



 The accuracy of our model is 0.79


## The model achieved a very good accuracy rate of close 80% predicting whether a new customer will stay with the firm or more likely to churn
